<a href="https://colab.research.google.com/github/luasampaio/automacao/blob/main/ntb_datasus_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Automatização do Processo de Download Datasus
- Criado em 03.23
- Luciana Sampaio


In [10]:
import requests
from datetime import datetime, timedelta
from ftplib import FTP
import urllib.error
import zipfile
import os

In [11]:
def obter_ultima_url_valida(mes_inicio=1, ano_inicio=2024):
    url_base = 'ftp://ftp.datasus.gov.br/cnes/BASE_DE_DADOS_CNES_{ano}{mes:02d}.ZIP'

    mes_atual, ano_atual = datetime.now().month, datetime.now().year
    data_inicio = datetime(ano_inicio, mes_inicio, 1)
    data_atual = datetime(ano_atual, mes_atual, 1)

    while data_inicio <= data_atual:
        ultimo_dia_do_mes = (data_inicio.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
        dias_no_mes = ultimo_dia_do_mes.day
        url = url_base.format(ano=data_inicio.year, mes=data_inicio.month)

        try:
            # Verifica se a URL existe e possui tamanho maior que 0
            tamanho_arquivo = obter_tamanho_arquivo_ftp(url)

            if tamanho_arquivo is not None and tamanho_arquivo > 0:
                ultima_url_valida = url

            else:
                break

        except urllib.error.URLError:
            # Trata erro de URL (por exemplo, 404 Not Found)
            break

        data_inicio += timedelta(days=dias_no_mes)  # Incrementa de acordo com os dias no mês

    return ultima_url_valida

In [12]:
def obter_tamanho_arquivo_ftp(url):
    try:
        # Extrai informações de URL
        partes_url = url.split('/')
        servidor_ftp = partes_url[2]
        caminho_arquivo = '/'.join(partes_url[3:])

        # Conecta-se ao servidor FTP
        with FTP(servidor_ftp) as ftp:
            ftp.login()

            tamanho_arquivo = ftp.size(caminho_arquivo)

        return tamanho_arquivo

    except Exception as e:
        return None

In [13]:
import re

def extrair_ano_mes_da_url(url):
    # Define um padrão regex para extrair o ano e o mês
    padrao = r'BASE_DE_DADOS_CNES_(\d{4})(\d{2})\.ZIP'

    # Tenta encontrar o padrão na URL
    correspondencia = re.search(padrao, url)

    if correspondencia:
        ano = correspondencia.group(1)
        mes = correspondencia.group(2)
        ano_mes = f"{ano}{mes}"
        return ano_mes
    else:
        return None

In [14]:
url_atual = obter_ultima_url_valida()

In [15]:
print(url_atual)

ftp://ftp.datasus.gov.br/cnes/BASE_DE_DADOS_CNES_202412.ZIP


In [16]:
ano_mes = extrair_ano_mes_da_url(url_atual)

In [17]:
ano_mes

'202412'